In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from eazyetl import Load

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")
city_name = 'Nairobi'
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"

In [3]:
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    city = data.get('name')
    temperature = data['main']['temp']
    humidity = data['main']['humidity']
    description = data['weather'][0]['description']
    print(description)
else:
    print(f"Error loading data: {response.status_code}")

light rain


In [4]:
data

{'coord': {'lon': 36.8167, 'lat': -1.2833},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 295.71,
  'feels_like': 295.64,
  'temp_min': 295.71,
  'temp_max': 295.71,
  'pressure': 1014,
  'humidity': 62,
  'sea_level': 1014,
  'grnd_level': 839},
 'visibility': 10000,
 'wind': {'speed': 2.98, 'deg': 128, 'gust': 2.92},
 'rain': {'1h': 0.6},
 'clouds': {'all': 97},
 'dt': 1746527142,
 'sys': {'country': 'KE', 'sunrise': 1746502043, 'sunset': 1746545476},
 'timezone': 10800,
 'id': 184745,
 'name': 'Nairobi',
 'cod': 200}

In [5]:
df = pd.DataFrame({
    'City': [city],
    'Description': [description],
    'Humidity': [humidity],
    'Temperature': [temperature]
}, )

df.head()

,City,Description,Humidity,Temperature
0,Nairobi,light rain,62,295.71


In [6]:
# Load our data into a PostgreSQL database

DB_CONN_URL=os.getenv("DB_CONN_URL")
# print(DB_CONN_URL)

engine = create_engine(f"{DB_CONN_URL}")
try:
    df.to_sql(name='nbo_data', con=engine, schema='public', if_exists='replace', index=True)
    connection = engine.connect()
    query = connection.execute(text('SELECT * FROM public.nbo_data LIMIT 5')) # test to see if the table is created.
    for row in query:
        print(row)
    connection.close()

except Exception as e:
    print(f"Error: {str(e)}")



(0, 'Nairobi', 'light rain', 62, 295.71)
